In [13]:
import numpy as np

folder = 'k-12_mg1655'
metabolites = np.load(f'{folder}/metabolites.npy', allow_pickle=True).item()

print(len(metabolites), 'metabolites')

1412 metabolites


In [120]:
# # this takes a while...

# meta_smiles = {}
# i, total = 0, len(metabolites)
# for m in list(metabolites):
#     i += 1
#     print(f'{i} / {total}      ',  end='\r')
#     cs = get_compounds(m, 'name')

#     def parse_c(c:Compound):
#         return c.cid, c.canonical_smiles, c.isomeric_smiles, c.charge,

#     result = list(map(lambda c: parse_c(c), cs))
#     if len(result) < 1: result = None
#     meta_smiles[m] = result

# print()
# print(len(meta_smiles))
# np.save(f'{folder}/meta_smiles', meta_smiles)

1412 / 1412      
1412


In [80]:
import numpy as np
meta_smiles = np.load(f'{folder}/meta_smiles.npy', allow_pickle=True).item()

rectified = {}
ambiguous = {}
for k, d in meta_smiles.items():
    num_mols = len(d) if d is not None else 0
    if num_mols < 2: continue
    
    all_good = list(range(num_mols))            

    tests = [
        lambda m: m[3] == 0, # neutral charge
        lambda m: not('+' in m[1] and '-' in m[1]), # favour uncharged notation
        lambda m: '.' not in m[1], # connected
    ]

    ti = 0
    while len(all_good) > 1 and ti < len(tests):
        all_same, first_smiles = True, d[all_good[0]][1]
        to_del = []
        test = tests[ti]
        for _i in range(len(all_good)-1, -1, -1):
            i = all_good[_i]
            all_same = all_same and d[i][1] == first_smiles
            if not test(d[i]): to_del.append(_i)
        
        if all_same and len(all_good) > 0:
            all_good = [all_good[0]]
            break
        elif len(to_del) < len(all_good): # if a test rules out everything, don't apply
            for _i in to_del: del all_good[_i]

        ti +=1


    def rectify(selected): # selected is array of 1
        meta_smiles[k] = selected
        rectified[k] = (selected, d)

    if len(all_good) == 1:
        rectify(d[all_good[0]])
    else:
        ambiguous[k] = d

print()
print(len(rectified), 'rectified, ', len(ambiguous), 'still ambiguous')

np.save(f'{folder}/meta_smiles_pp1', meta_smiles)
np.save(f'{folder}/meta_smiles_pp1_rectified', rectified)
np.save(f'{folder}/meta_smiles_pp1_ambiguous', ambiguous)


122 rectified 1 still ambiguous


In [ ]:
rectified = {}

In [14]:
import numpy as np
meta_smiles = np.load(f'{folder}/meta_smiles_pp1.npy', allow_pickle=True).item()

ambiguous = {}
nones = list(map(lambda x: x[0], filter(lambda x: x[1] is None, meta_smiles.items())))
total = len(nones)
print(total, 'Nones')

# index is ghetto pass by ref
def cut(string:str, start, end, index=None):
    i = 0 if index is None else index[0]
    try:
        s = string.index(start, i)
        e = len(string) if end is None else string.index(end, s+len(start)) + len(end)
        if not index is None: index[0] = e
        return string[s:e]
    except ValueError:
        return None

exit = False
for name in nones:
    for a in ['an ', 'a ']:
        if name.startswith(a):
            name = name[len(a):]

    cut_i = [0]
    while True:
        seg = cut(name, '[', ']', cut_i)
        if seg is None: break

        print(seg)
        exit = True

    if exit: 
        print(name)
        break

839 Nones
[glycine-cleavage complex H protein]
[(R)-lipoyl]
[glycine-cleavage complex H protein] N^(6)-[(R)-lipoyl]-L-lysine


In [26]:
# https://www.ebi.ac.uk/chebi/webServices.do

from zeep.client import Client

client = Client('https://www.ebi.ac.uk/webservices/chebi/2.0/webservice?wsdl')
x = client.service.getLiteEntity('N(6)-[(R)-lipoyl]-L-lysine', 'ALL', 50, 'ALL')

Forcing soap:address location to HTTPS


In [27]:
for xx in x:
    if '6' in xx['chebiAsciiName']:
        print(xx)

{
    'chebiId': 'CHEBI:83099',
    'chebiAsciiName': 'N(6)-[(R)-lipoyl]-L-lysine residue',
    'searchScore': 0.1,
    'entityStar': 3
}


scratch after this

In [28]:
x = client.service.getCompleteEntity('CHEBI:83099')

In [31]:
x['smiles']

'*-N[C@@H](CCCCNC(=O)CCCC[C@@H]1CCSS1)C(-*)=O'

In [105]:
x[0].canonical_smiles

'C1CSSC1CCCCC(=O)NCCCCC(C(=O)O)N'

In [90]:
dg = get_compounds('D-glucose', 'name')[0]
tdp = get_compounds('dTDP', 'name')[0]

In [92]:
print(dg.canonical_smiles)
print(tdp.canonical_smiles)

C(C1C(C(C(C(O1)O)O)O)O)O
CC1=CN(C(=O)NC1=O)C2CC(C(O2)COP(=O)(O)OP(=O)(O)O)O


In [94]:
x = get_compounds('dTDP-D-glucose', 'name')[0]

In [95]:
x.canonical_smiles

'CC1=CN(C(=O)NC1=O)C2CC(C(O2)COP(=O)(O)OP(=O)(O)OC3C(C(C(C(O3)CO)O)O)O)O'

In [8]:
cs2d = get_compounds('6-phospho D-glucono-1,5-lactone', 'name', record_type='2d')

In [11]:
cs2d[0]

Compound(46926335)

In [ ]:
cs